# ZeroMQ Interface

In [ ]:
from threading import Thread
from time import sleep
from profit.run.zeromq import ZeroMQInterface, ZeroMQRunnerInterface
from profit.config import BaseConfig, ZeroMQInterfaceConfig

BASE_CONFIG = BaseConfig.from_file('numpy.yaml')
MAX_IDS = BASE_CONFIG['ntrain']
config = {'class': 'zeromq'}
config = ZeroMQInterfaceConfig(**config)
config.process_entries(BASE_CONFIG)

def runner():
    rif = ZeroMQRunnerInterface(config, MAX_IDS, BASE_CONFIG['input'], BASE_CONFIG['output'])
    print(f'input_vars {rif.input_vars}')
    print(f'output_vars {rif.output_vars}')
    rif.input[['u', 'v']][1] = 1, -1
    for i in range(3):
        rif.poll()
        sleep(0.5)
        print(f'runner {i}')
    print(f'output {rif.output}')
    print(f'internal {rif.internal}')
    rif.clean()

def worker():
    wif = ZeroMQInterface(config, 1)
    print(f'input {wif.input}')
    wif.output['f'] = 2 * wif.input['u']
    wif.time = 2
    wif.done()

rt = Thread(target=runner)
wt = Thread(target=worker)

rt.start()
wt.start()
wt.join()
rt.join()